In [1]:
import os
import pandas as pd
from datetime import date, timedelta
from functools import reduce
import operator
from dateutil.relativedelta import relativedelta
import datetime
from datetime import datetime
import numpy as np
import warnings
warnings.filterwarnings("ignore") 

In [3]:
Availability = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\ServiceVisibility_Availability_Inc.csv',index_col=False,header=0,sep='|')
domain_map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\model1_mapping.csv')

In [4]:
list(Availability)

['assignment_group_current_domain',
 'business_service_name',
 'opened_at',
 'state',
 'major_incident_state',
 'promoted_on',
 'business_duration',
 'priority']

In [47]:
#domain_map = domain_map.loc[domain_map.notnull().sum(1).groupby(domain_map.Name).idxmax()]

In [48]:
sva_1 = []
sva_2 = []
sva_3 = []
sva_4 = []
sva_5 = []
sva_6 = []

In [49]:
today = date.today()
days_before = (date.today()-timedelta(days=30))
days_before2 = (date.today()-timedelta(days=182))
days_before3 = (date.today()-timedelta(days=10))
days_after = (date.today()+timedelta(days=30))
year_start=pd.to_datetime(str((date.today()- relativedelta(years=1)).year)+'-10-01',format='%Y-%m-%d')

In [50]:
Availability['business_service_name'] = Availability['business_service_name'].str.strip('[PROD]')
Availability['business_service_name'] = Availability['business_service_name'].str.strip()

In [51]:
Availability['opened_at'] = pd.to_datetime(Availability['opened_at'],format='%Y-%m-%d')
Availability['opened_at'] = Availability['opened_at'].apply(lambda x: x.strftime('%Y-%m-%d'))
Availability['opened_at']= pd.to_datetime(Availability['opened_at'],format='%Y-%m-%d')

In [52]:
#Availability = Availability.merge(domain_map,how='left',left_on=['business_service_name'],right_on=['Name'])

In [53]:
sva1 = Availability[['assignment_group_current_domain','business_service_name','opened_at','state']]
sva1 = sva1.loc[(sva1['state'].isin(['NEW','IN PROGRESS','ON HOLD'])) &  (sva1['opened_at'] < days_before2) & (sva1['assignment_group_current_domain'].isin(['AUSTRALIA RETAIL AND COMMERCIAL','BANKING SERVICES','EMPLOYEE EXPERIENCE','INFRASTRUCTURE AS A SERVICE','INSTITUTIONAL','NEW ZEALAND','SECURITY','TECHNOLOGY OPERATIONS','WEALTH']))]
sva1['AgedIncidentsOpenTotal_Text'] = 'Aged Incident Open Total (greater than 6 months)'
sva1_data=sva1.groupby(['assignment_group_current_domain','business_service_name','AgedIncidentsOpenTotal_Text'])['state'].count().reset_index(name='AgedIncidentsOpenTotal')

In [54]:
sva_1.append(sva1_data)

In [55]:
sva2 = Availability[['assignment_group_current_domain','business_service_name','promoted_on','state','major_incident_state']]
sva2 = sva2.loc[(sva2['major_incident_state'].isin(['accepted']))]
sva2['promoted_on'] = pd.to_datetime(sva2['promoted_on'],format='%Y-%m-%d')
sva2['promoted_on'] = sva2['promoted_on'].apply(lambda x: x.strftime('%Y-%m-%d'))
sva2['promoted_on']= pd.to_datetime(sva2['promoted_on'],format='%Y-%m-%d')
sva2 = sva2.loc[(sva2['state'].isin(['NEW','IN PROGRESS','ON HOLD'])) &  (sva2['promoted_on'] > days_before3) & (sva2['assignment_group_current_domain'].isin(['AUSTRALIA RETAIL AND COMMERCIAL','BANKING SERVICES','EMPLOYEE EXPERIENCE','INFRASTRUCTURE AS A SERVICE','INSTITUTIONAL','NEW ZEALAND','SECURITY','TECHNOLOGY OPERATIONS','WEALTH']))]
sva2['MajorIncidents_Lt10Days_Text'] = 'Major incidents Open'
sva2_data=sva2.groupby(['assignment_group_current_domain','business_service_name','MajorIncidents_Lt10Days_Text'])['state'].count().reset_index(name='MajorIncidents_Lt10Days')

In [56]:
sva_2.append(sva2_data)

In [57]:
sva3 = Availability[['assignment_group_current_domain','business_service_name','opened_at','state']]
sva3 = sva3[(sva3['state'].isin(['NEW','IN PROGRESS','ON HOLD'])) &  ((sva3['opened_at'] < days_before3) & (sva3['opened_at'] >= days_before))  & (sva3['assignment_group_current_domain'].isin(['AUSTRALIA RETAIL AND COMMERCIAL','BANKING SERVICES','EMPLOYEE EXPERIENCE','INFRASTRUCTURE AS A SERVICE','INSTITUTIONAL','NEW ZEALAND','SECURITY','TECHNOLOGY OPERATIONS','WEALTH']))]
sva3['MajorIncidents_Lt30Days_Text'] = 'all incidents open in last 30 days'
sva3_data=sva3.groupby(['assignment_group_current_domain','business_service_name','MajorIncidents_Lt30Days_Text'])['opened_at'].count().reset_index(name='MajorIncidents_Lt30Days')

In [58]:
sva_3.append(sva3_data)

In [59]:
sva4 = Availability[['assignment_group_current_domain','business_service_name','opened_at','state']]
sva4 = sva4.loc[(sva4['state'].isin(['NEW','IN PROGRESS','ON HOLD'])) &  (sva4['opened_at'] < days_before)  & (sva4['assignment_group_current_domain'].isin(['AUSTRALIA RETAIL AND COMMERCIAL','BANKING SERVICES','EMPLOYEE EXPERIENCE','INFRASTRUCTURE AS A SERVICE','INSTITUTIONAL','NEW ZEALAND','SECURITY','TECHNOLOGY OPERATIONS','WEALTH']))]
sva4['MajorIncidents_Gt30Days_Text'] = 'all incidents > 30 days'
sva4_data=sva4.groupby(['assignment_group_current_domain','business_service_name','MajorIncidents_Gt30Days_Text'])['opened_at'].count().reset_index(name='MajorIncidents_Gt30Days')

In [60]:
sva_4.append(sva4_data)

In [61]:
sva5 = Availability[['assignment_group_current_domain','business_service_name','promoted_on','business_duration','major_incident_state']]
sva5 = sva5.loc[(sva5['major_incident_state'].isin(['accepted']))]
sva5['promoted_on'] = pd.to_datetime(sva5['promoted_on'],format='%Y-%m-%d')
sva5['promoted_on'] = sva5['promoted_on'].apply(lambda x: x.strftime('%Y-%m-%d'))
sva5['promoted_on']= pd.to_datetime(sva5['promoted_on'],format='%Y-%m-%d')
sva5 = sva5.loc[ (sva5['business_duration'] > 0) &  (sva5['promoted_on'] >= year_start)  & (sva5['assignment_group_current_domain'].isin(['AUSTRALIA RETAIL AND COMMERCIAL','BANKING SERVICES','EMPLOYEE EXPERIENCE','INFRASTRUCTURE AS A SERVICE','INSTITUTIONAL','NEW ZEALAND','SECURITY','TECHNOLOGY OPERATIONS','WEALTH']))]
sva5['MajorIncidents_Text'] = 'Major Incidents'
sva5_data=sva5.groupby(['assignment_group_current_domain','business_service_name','MajorIncidents_Text'])['promoted_on'].count().reset_index(name='MajorIncidents')

In [62]:
sva_5.append(sva5_data)

In [63]:
sva6 = Availability[['assignment_group_current_domain','business_service_name','opened_at','business_duration','major_incident_state']]
sva6 = sva6.loc[sva6['major_incident_state']!='accepted']
sva6 = sva6.loc[(sva6['business_duration'] > 0) &  (sva6['opened_at'] >= year_start)  & (sva6['assignment_group_current_domain'].isin(['AUSTRALIA RETAIL AND COMMERCIAL','BANKING SERVICES','EMPLOYEE EXPERIENCE','INFRASTRUCTURE AS A SERVICE','INSTITUTIONAL','NEW ZEALAND','SECURITY','TECHNOLOGY OPERATIONS','WEALTH']))]
sva6['OtherIncidents_Text'] = 'Other Incidents'
sva6_data=sva6.groupby(['assignment_group_current_domain','business_service_name','OtherIncidents_Text'])['opened_at'].count().reset_index(name='OtherIncidents')

In [64]:
sva_6.append(sva6_data)

In [65]:
sva_Availability1=reduce(operator.concat, sva_1)
sva_Availability2=reduce(operator.concat, sva_2) 
sva_Availability3=reduce(operator.concat, sva_3) 
sva_Availability4=reduce(operator.concat, sva_4) 
sva_Availability5=reduce(operator.concat, sva_5)
sva_Availability6=reduce(operator.concat, sva_6)

In [66]:
sva_Availability1 = pd.DataFrame(sva_Availability1)
sva_Availability2 = pd.DataFrame(sva_Availability2)
sva_Availability3 = pd.DataFrame(sva_Availability3)
sva_Availability4 = pd.DataFrame(sva_Availability4)
sva_Availability5 = pd.DataFrame(sva_Availability5)
sva_Availability6 = pd.DataFrame(sva_Availability6)

In [67]:
dfs = [sva_Availability1,sva_Availability2,sva_Availability3,sva_Availability4,sva_Availability5,sva_Availability6]

In [68]:
sva_Availability_final = reduce(lambda left,right: pd.merge(left,right,on=['assignment_group_current_domain','business_service_name'],how='outer'),dfs)

In [69]:
sva_Availability_final = pd.DataFrame(sva_Availability_final)

In [70]:
sva_Availability_final['AgedIncidentsOpenTotal_Text'] = sva_Availability_final['AgedIncidentsOpenTotal_Text'].fillna('Aged Incident Open Total (greater than 6 months)')
sva_Availability_final['MajorIncidents_Lt10Days_Text'] = sva_Availability_final['MajorIncidents_Lt10Days_Text'].fillna('Major incidents Open')
sva_Availability_final['MajorIncidents_Lt30Days_Text'] = sva_Availability_final['MajorIncidents_Lt30Days_Text'].fillna('all incidents open in last 30 days')
sva_Availability_final['MajorIncidents_Gt30Days_Text'] = sva_Availability_final['MajorIncidents_Gt30Days_Text'].fillna('all incidents > 30 days')
sva_Availability_final['MajorIncidents_Text'] = sva_Availability_final['MajorIncidents_Text'].fillna('Major Incidents')
sva_Availability_final['OtherIncidents_Text'] = sva_Availability_final['OtherIncidents_Text'].fillna('Other Incidents')
sva_Availability_final = sva_Availability_final.fillna(0)

In [71]:
sva_Availability_final['AgedIncidentsOpenTotal'] = sva_Availability_final['AgedIncidentsOpenTotal'].astype(int)
sva_Availability_final['MajorIncidents_Lt10Days'] = sva_Availability_final['MajorIncidents_Lt10Days'].astype(int)
sva_Availability_final['MajorIncidents_Lt30Days'] = sva_Availability_final['MajorIncidents_Lt30Days'].astype(int)
sva_Availability_final['MajorIncidents_Gt30Days'] = sva_Availability_final['MajorIncidents_Gt30Days'].astype(int)
sva_Availability_final['MajorIncidents'] = sva_Availability_final['MajorIncidents'].astype(int)
sva_Availability_final['OtherIncidents'] = sva_Availability_final['OtherIncidents'].astype(int)

In [72]:
sva_Availability_final = sva_Availability_final[['assignment_group_current_domain','business_service_name','AgedIncidentsOpenTotal_Text','AgedIncidentsOpenTotal','MajorIncidents_Lt10Days_Text','MajorIncidents_Lt10Days','MajorIncidents_Lt30Days_Text','MajorIncidents_Lt30Days','MajorIncidents_Gt30Days_Text','MajorIncidents_Gt30Days','MajorIncidents_Text','MajorIncidents','OtherIncidents_Text','OtherIncidents']]

In [73]:
sva_Availability_final =sva_Availability_final.drop_duplicates(keep='first')

In [74]:
sva_Availability_final=sva_Availability_final.rename(columns={'assignment_group_current_domain':'DomainINC'})

In [75]:
sva_Availability_final=sva_Availability_final.rename(columns={'Domain':'DomainINC'})

In [76]:
sva_Availability_final

,DomainINC,business_service_name,AgedIncidentsOpenTotal_Text,AgedIncidentsOpenTotal,MajorIncidents_Lt10Days_Text,MajorIncidents_Lt10Days,MajorIncidents_Lt30Days_Text,MajorIncidents_Lt30Days,MajorIncidents_Gt30Days_Text,MajorIncidents_Gt30Days,MajorIncidents_Text,MajorIncidents,OtherIncidents_Text,OtherIncidents
0,AUSTRALIA RETAIL AND COMMERCIAL,ACLE DATABASE SUPPORT,Aged Incident Open Total (greater than 6 months),2,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,2,Major Incidents,0,Other Incidents,0
1,AUSTRALIA RETAIL AND COMMERCIAL,ACTIVE DIRECTORY - CORPTST.ANZ.COM,Aged Incident Open Total (greater than 6 months),1,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,1,Major Incidents,0,Other Incidents,0
2,AUSTRALIA RETAIL AND COMMERCIAL,AMBIT ASSET FINANCE - AU,Aged Incident Open Total (greater than 6 months),3,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,11,Major Incidents,0,Other Incidents,653
3,AUSTRALIA RETAIL AND COMMERCIAL,ANZ APP - AU,Aged Incident Open Total (greater than 6 months),2,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,8,Major Incidents,2,Other Incidents,512
4,AUSTRALIA RETAIL AND COMMERCIAL,ANZ COMMISSIONS PLATFORM [EXT,Aged Incident Open Total (greater than 6 months),5,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,5,Major Incidents,0,Other Incidents,0
5,AUSTRALIA RETAIL AND COMMERCIAL,ANZ SHARE INVESTING,Aged Incident Open Total (greater than 6 months),5,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,44,Major Incidents,0,Other Incidents,67
6,AUSTRALIA RETAIL AND COMMERCIAL,ANZ SHARE INVESTING MOBILE - AU,Aged Incident Open Total (greater than 6 months),1,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,1,Major Incidents,0,Other Incidents,0
7,AUSTRALIA RETAIL AND COMMERCIAL,ATA ACCESS GOVERNANCE REPORTING,Aged Incident Open Total (greater than 6 months),1,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,1,Major Incidents,0,Other Incidents,0
8,AUSTRALIA RETAIL AND COMMERCIAL,AZURE ACTIVE DIRECTORY - SYNCING SERVICE,Aged Incident Open Total (greater than 6 months),1,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,1,Major Incidents,0,Other Incidents,0
9,AUSTRALIA RETAIL AND COMMERCIAL,BACKU,Aged Incident Open Total (greater than 6 months),2,Major incidents Open,0,all incidents open in last 30 days,0,all incidents > 30 days,2,Major Incidents,0,Other Incidents,1


In [77]:
#sva_Availability_final.loc[(sva_Availability_final['business_service_name'] == 'V2PLUS')]

In [78]:
domain_map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\model1_mapping.csv')

In [79]:
domain_map = domain_map.loc[domain_map.notnull().sum(1).groupby(domain_map.Name).idxmax()]

In [80]:
fdata = domain_map[['Name','Domain']]

In [81]:
fdata

,Name,Domain
17,ANZ GameSaveMatch Website [Ext],Australia Retail and Commercial
16,ANZ Mobile Lending Website [Ext],Australia Retail and Commercial
10,ANZ Shield App Credential [Ext],Australia Retail and Commercial
9,ANZ Shield Credential,Australia Retail and Commercial
6,AU CSP - CAZ BFF [API],Australia Retail and Commercial
4,Adobe Experience Manager - Electronic Forms,Australia Retail and Commercial
11,Anz.com - eAdmin,Australia Retail and Commercial
13,Business Benchmarking Calculators [Ext],Australia Retail and Commercial
5,CAZ (Commercial A to Z - Desktop) - AU,Australia Retail and Commercial
0,EFT Disputes - AU,Australia Retail and Commercial


In [82]:
fdata['Domain']=fdata['Domain'].str.upper()
fdata['Name']=fdata['Name'].str.upper()

In [83]:
f = sva_Availability_final.merge(fdata,how='left',left_on=['business_service_name','DomainINC'],right_on=['Name','Domain'])

In [84]:
f1 = f[~(f['Name'].isnull())]

In [85]:
len(f1)

10

In [86]:
f1 = f1 [['DomainINC','business_service_name','AgedIncidentsOpenTotal','MajorIncidents_Lt10Days','MajorIncidents_Lt30Days','MajorIncidents_Gt30Days','MajorIncidents','OtherIncidents']]

In [87]:
f1.to_csv(r'C:\NotBackedUp\SOE\pyscript\ServiceNow\CSV\IncidentMetrics_15_04_2021.csv',index=None)

In [88]:
f1

,DomainINC,business_service_name,AgedIncidentsOpenTotal,MajorIncidents_Lt10Days,MajorIncidents_Lt30Days,MajorIncidents_Gt30Days,MajorIncidents,OtherIncidents
269,AUSTRALIA RETAIL AND COMMERCIAL,CAZ (COMMERCIAL A TO Z - DESKTOP) - AU,0,0,0,1,0,2
302,AUSTRALIA RETAIL AND COMMERCIAL,MAUDE - AU,0,0,0,4,1,129
826,AUSTRALIA RETAIL AND COMMERCIAL,ADOBE EXPERIENCE MANAGER - ELECTRONIC FORMS,0,0,0,0,0,36
835,AUSTRALIA RETAIL AND COMMERCIAL,ANZ MOBILE LENDING WEBSITE [EXT],0,0,0,0,0,4
838,AUSTRALIA RETAIL AND COMMERCIAL,ANZ SHIELD CREDENTIAL,0,0,0,0,0,19
843,AUSTRALIA RETAIL AND COMMERCIAL,ANZ.COM - EADMIN,0,0,0,0,0,14
844,AUSTRALIA RETAIL AND COMMERCIAL,ANZ.COM LEGACY (ASP),0,0,0,0,0,6
895,AUSTRALIA RETAIL AND COMMERCIAL,EFT DISPUTES - AU,0,0,0,0,0,2
939,AUSTRALIA RETAIL AND COMMERCIAL,HOST NAVIGATOR - AU,0,0,0,0,0,23
955,AUSTRALIA RETAIL AND COMMERCIAL,LOCATION INTELLIGENCE PLATFORM (LIP),0,0,0,0,0,1
